In [1]:
import requests, json, time
import polyline, folium
import datetime
import re

In [7]:
with open('config.json') as f:
    config = json.load(f)
    key = config['apiKey']
    
mode = 'bus'
origin = 'Sieradenbuurt'
destination = 'OOstvaardersbuurt'
departure = "11/03/2022 14:48"
arrival = None
   
def map_start(a, b):
    return ((a[0] + b[0]) / 2, (a[1] + b[1]) / 2)
    
def convert_unix(date):
    if date != None:
        return int(time.mktime(datetime.datetime.strptime(date, "%d/%m/%Y %H:%M").timetuple()))
    else:
        return None
    
def json_route(origin, destination, departure, arrival):
    url = 'https://maps.googleapis.com/maps/api/directions/json?'
    params = {
        'mode': mode.lower(),
        'origin': origin,
        'destination':destination, 
        'region':'nl',
        'key': key
    }
    if departure != None:
        params['departure_time'] = convert_unix(departure)
    
    response = requests.post(url, params=params)
    if response.status_code == 200:
        return json.loads(response.text)
    else:
        return {}

def get_coordinates(raw_json):
    if len(raw_json) > 0:
        coordinates = polyline.decode(raw_json['routes'][0]['overview_polyline']['points'], 5)
        return coordinates
    else:
        return False

def make_map(coordinates):
    if coordinates != False:
        start_coord = coordinates[0]
        end_coord = coordinates[len(coordinates) - 1]

        map = folium.Map(location=map_start(start_coord, end_coord),
                    zoom_start=15)
        
        folium.PolyLine(coordinates,
                        color='red',
                        weight=5,
                        opacity=0.8).add_to(map)
        
        folium.Marker(location=start_coord, popup=f'Start Location - <b>{origin}</b>', tooltip=f'<strong>{origin}</strong>', icon=folium.Icon(color='red', icon='map-pin', prefix = 'fa')).add_to(map)
        folium.Marker(location=end_coord, popup=f'End Location - <b>{destination}</b>', tooltip=f'<strong>{destination}</strong>', icon=folium.Icon(color='red', icon='flag-checkered', prefix = 'fa')).add_to(map)
        map.save("map.html")
        display(map)
    else:
        return "Could not process map"
        
def process_routing_info(raw_json):
    steps = []
    distance = raw_json['routes'][0]['legs'][0]['distance']['value']
    duration = raw_json['routes'][0]['legs'][0]['duration']['text']
    
    if mode in ['transit']:
        departure_time = raw_json['routes'][0]['legs'][0]['departure_time']['text']
        arrival_time = raw_json['routes'][0]['legs'][0]['arrival_time']['text']
        times = (departure_time, arrival_time)
        print(f"Planned departure: {departure_time}\nPlanned arrival: {arrival_time}")
    else:
        times = None
    
    if distance >= 1000:
        distance = round(distance/1000, 2)
        print('Distance: {} km'.format(distance))
    else:
        print('Distance {} m'.format(distance))
    print(f"Duration: {duration}")
    
    steps = [re.sub('<[^<]+?>', '', step['html_instructions'].replace('<div style="font-size:0.9em">', '\n')) for step in raw_json['routes'][0]['legs'][0]['steps']]
    for step in range(len(steps)):
        print(f"{step+1}. {steps[step]}")
    return [distance, duration, steps, times]

def main():
    raw_json = json_route(origin, destination, departure, arrival)
    #display(raw_json)
    coordinates = get_coordinates(raw_json)
    make_map(coordinates)
    route_info = process_routing_info(raw_json)
    
main()


Distance: 2.3 km
Duration: 6 mins
1. Head northeast on Sieradenweg toward Parurestraat
2. Turn right onto Bintangweg/Sieradenweg
3. Turn right onto Evenaar
4. At the roundabout, take the 1st exit
5. Continue onto Niasstraat
6. Continue onto Reijgersdaalstraat
7. Turn left onto Laan der V.O.C.
8. Turn right onto Cornelis Matelieffstraat
Destination will be on the right


## See monuments

In [22]:
url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

params = {
        'location':'52.3791,4.9003', 
        'radius': '30000',
        'type': 'restaurant',
        'key': key
    }

response = requests.get(url, params=params)
json_response = json.loads(response.text)
location_dict = {}
for place in json_response['results']:
    location_dict[place['name']] = list(place['geometry']['location'].values())
print(location_dict)

{'Hotel ibis Amsterdam Centre': [52.3796129, 4.8971342], 'Park Plaza Victoria Amsterdam': [52.37709, 4.8979799], 'INK Hotel Amsterdam - MGallery': [52.3754828, 4.8930435], 'Radisson Blu Hotel, Amsterdam City Center': [52.3702272, 4.8967209], 'Hotel ibis Amsterdam Centre Stopera': [52.3700867, 4.907651], 'Hotel Torenzicht': [52.3725361, 4.8981806], 'Hotel Swissôtel Amsterdam': [52.37361019999999, 4.8933274], 'Grand Hotel Amrâth Amsterdam': [52.37458710000001, 4.903903199999999], "Belushi's": [52.373999, 4.8962872], 'Die Port Van Cleve': [52.3740512, 4.8906021], 'Hotel Old Nickel': [52.37634569999999, 4.899674699999998], 'Hotel Prins Hendrik': [52.37652749999999, 4.8995948], "art'otel amsterdam": [52.37764, 4.89716], 'Hotel V': [52.3708421, 4.893662900000001], 'Hotel Doria': [52.3724415, 4.8943599], 'Hotel Jimmy': [52.3777175, 4.896180500000001], 'Vapiano Amsterdam Oosterdokskade': [52.3758926, 4.9083771], 'De Belhamel': [52.3794166, 4.892049], 'Mata Hari': [52.3746613, 4.90013], 'Winkel